In [1]:
import pymongo as pm
import pandas as pd
import numpy as np
import regex as re
import unicodedata

In [2]:
# Connexion a la database de type MongoDB
client = pm.MongoClient()
rawData = client.rawDB
rawCollection = rawData['products']

# Comptage du nombre de documents
rawCollection.estimated_document_count()

1601557

In [3]:
# Pour reduire le nombre de documents, on ne prends que ceux vendus en France
#rawCollection.count_documents({'countries':'France'})

In [4]:
# Exemple de document pour avoir une idee des informations disponibles
# rawCollection.find_one({'countries':'France'})

In [5]:
## On va creer des tableaux pandas a partir des informations de la base de donnees
# D'abord pour les noms
cursor = rawCollection.find({'countries':'France', 'manufacturing_places': { "$exists": True}},
                       {"ecoscore_data.agribalyse.name_fr":1,
                        "product_name_fr":1,
                        "product_name":1,
                        "generic_name_fr":1,
                        "generic_name":1,
                       })

name_df = pd.DataFrame(list(cursor))

In [6]:
# Uniformisation de la colonne ecoscore_data

for i in range(len(name_df)):
    if type(name_df.loc[i,'ecoscore_data']) == dict:
        if name_df.loc[i,'ecoscore_data']['agribalyse']:
            name_df.loc[i,'ecoscore_data'] = name_df.loc[i,'ecoscore_data']['agribalyse']['name_fr']
        else:
            name_df.loc[i,'ecoscore_data'] = np.nan
    else:
        continue

In [7]:
# Uniformisation des noms

for i in range(len(name_df)) :
    if type(name_df.loc[i,'generic_name_fr']) == str and len(name_df.loc[i,'generic_name_fr']) == 0:
        name_df.loc[i,'generic_name_fr'] = np.nan
    elif type(name_df.loc[i,'generic_name']) == str and len(name_df.loc[i,'generic_name']) == 0:
        name_df.loc[i,'generic_name'] = np.nan
    elif type(name_df.loc[i,'product_name_fr']) == str and len(name_df.loc[i,'product_name_fr']) == 0:
        name_df.loc[i,'product_name_fr'] = np.nan
    elif type(name_df.loc[i,'product_name']) == str and len(name_df.loc[i,'product_name']) == 0:
        name_df.loc[i,'product_name'] = np.nan
    elif type(name_df.loc[i,'ecoscore_data']) == str and len(name_df.loc[i,'ecoscore_data']) == 0:
        name_df.loc[i,'ecoscore_data'] = np.nan
    else:
        continue

In [8]:
# Puis le reste des informtations

cursor = rawCollection.find({'countries':'France', 'manufacturing_places': { "$exists": True}},
                       {"code":1,
                        "countries":1,
                        "nutriscore_grade":1,
                        "nutriscore_score":1,
                        "ecoscore_grade":1,
                        "ecoscore_score":1,
                        "manufacturing_places":1,
                        "brands":1
                       })

main_df = pd.DataFrame(list(cursor))

In [9]:
# Nouveau jeu de donnees plus clair

neatMain_df = pd.DataFrame(columns=['Code Barre','Nom du Produit','Endroit de Manufacture','Categorie Principale','Categorie Secondaire','Categorie Tertiaire','Nutriscore','Lettre Nutriscore','Ecoscore','Lettre Ecoscore','Marque de Distribution'])
neatMain_df['Code Barre'] = main_df['code']

for i in range(len(name_df)):
    if pd.isnull(name_df.loc[i,'ecoscore_data']) == False:
        neatMain_df.loc[i,'Nom du Produit'] = name_df.loc[i,'ecoscore_data']
    else :
        if pd.isnull(name_df.loc[i,'product_name_fr']) == False:
            neatMain_df.loc[i,'Nom du Produit'] = name_df.loc[i,'product_name_fr']
        else : 
            if pd.isnull(name_df.loc[i,'generic_name_fr']) == False:
                neatMain_df.loc[i,'Nom du Produit'] = name_df.loc[i,'generic_name_fr']
            else :
                if pd.isnull(name_df.loc[i,'product_name']) == False:
                    neatMain_df.loc[i,'Nom du Produit'] = name_df.loc[i,'product_name']
                else :
                    if pd.isnull(name_df.loc[i,'generic_name']) == False:
                        neatMain_df.loc[i,'Nom du Produit'] = name_df.loc[i,'generic_name']
                    else :
                        neatMain_df.loc[i,'Nom du Produit'] = np.nan

neatMain_df['Endroit de Manufacture'] = main_df['manufacturing_places']

for i in range(len(main_df)):
    if type(main_df.loc[i,'categories']) == str:
        if len(main_df.loc[i,'categories'].split(',')) == 0:
            neatMain_df.loc[i,'Categorie Principale'] = np.nan
            neatMain_df.loc[i,'Categorie Secondaire'] = np.nan
            neatMain_df.loc[i,'Categorie Tertiaire'] = np.nan
        elif len(main_df.loc[i,'categories'].split(',')) == 1:
            neatMain_df.loc[i,'Categorie Principale'] = main_df.loc[i,'categories'].split(',')[0]
            neatMain_df.loc[i,'Categorie Secondaire'] = np.nan
            neatMain_df.loc[i,'Categorie Tertiaire'] = np.nan
        elif len(main_df.loc[i,'categories'].split(',')) == 2:
            neatMain_df.loc[i,'Categorie Principale'] = main_df.loc[i,'categories'].split(',')[0]
            neatMain_df.loc[i,'Categorie Secondaire'] = main_df.loc[i,'categories'].split(',')[1]
            neatMain_df.loc[i,'Categorie Tertiaire'] = np.nan
        else:
            neatMain_df.loc[i,'Categorie Principale'] = main_df.loc[i,'categories'].split(',')[0]
            neatMain_df.loc[i,'Categorie Secondaire'] = main_df.loc[i,'categories'].split(',')[1]
            neatMain_df.loc[i,'Categorie Tertiaire'] = main_df.loc[i,'categories'].split(',')[2]
    else:
        neatMain_df.loc[i,'Categorie Principale'] = np.nan
        neatMain_df.loc[i,'Categorie Secondaire'] = np.nan
        neatMain_df.loc[i,'Categorie Tertiaire'] = np.nan

neatMain_df['Lettre Nutriscore'] = main_df['nutriscore_grade'].str.upper()
neatMain_df['Lettre Ecoscore'] = main_df['ecoscore_grade'].str.upper()
neatMain_df['Lettre Nutriscore'] = neatMain_df['Lettre Nutriscore'].replace('UNKNOWN', np.nan)
neatMain_df['Lettre Ecoscore'] = neatMain_df['Lettre Ecoscore'].replace('UNKNOWN', np.nan)

neatMain_df['Nutriscore'] = main_df['nutriscore_score']
neatMain_df['Ecoscore'] = main_df['ecoscore_score']

neatMain_df['Marque de Distribution'] = main_df['brands']

neatMain_df.head()

,Code Barre,Nom du Produit,Endroit de Manufacture,Categorie Principale,Categorie Secondaire,Categorie Tertiaire,Nutriscore,Lettre Nutriscore,Ecoscore,Lettre Ecoscore,Marque de Distribution
0,30383354190402,"Sauce tomate aux oignons, préemballée",,Epicerie,Sauces,Sauces tomate,2.0,B,78.417014,B,panzani
1,00,lignaform,,Snacks,Snacks sucrés,Biscuits et gâteaux,8.0,C,NaN,NaN,lignaform
2,0000000000100,Moutarde,,Epicerie,Condiments,Sauces,18.0,D,58.949199,C,courte paille
3,0000000001199,Solène céréales poulet,,,NaN,NaN,NaN,NaN,NaN,NaN,CROUS
4,0000000001281,Tarte noix de coco,,Tartes,Tartes sucrées,Tartes à la noix de coco,14.0,D,NaN,NaN,"Crous Resto',Crous"


In [10]:
# Suppression des produits sans aucun score

noScore = 0
deletedIndex = []
droppedMain_df = neatMain_df

for i in range(len(droppedMain_df)):
    if pd.isnull(droppedMain_df.loc[i,'Lettre Nutriscore']):
        if pd.isnull(droppedMain_df.loc[i,'Lettre Ecoscore']):
            if pd.isnull(droppedMain_df.loc[i,'Nutriscore']):
                if pd.isnull(droppedMain_df.loc[i,'Ecoscore']):
                    deletedIndex.append(droppedMain_df.index[[i]].tolist()[0])
                    noScore += 1

droppedMain_df = droppedMain_df.drop(droppedMain_df.index[deletedIndex])

print("%s produits n'avaient aucun score et ont donc ete supprimes. Il reste %s produits." %(noScore,len(droppedMain_df)))

droppedMain_df.head()

27077 produits n'avaient aucun score et ont donc ete supprimes. Il reste 166670 produits.


,Code Barre,Nom du Produit,Endroit de Manufacture,Categorie Principale,Categorie Secondaire,Categorie Tertiaire,Nutriscore,Lettre Nutriscore,Ecoscore,Lettre Ecoscore,Marque de Distribution
0,30383354190402,"Sauce tomate aux oignons, préemballée",,Epicerie,Sauces,Sauces tomate,2.0,B,78.417014,B,panzani
1,00,lignaform,,Snacks,Snacks sucrés,Biscuits et gâteaux,8.0,C,NaN,NaN,lignaform
2,0000000000100,Moutarde,,Epicerie,Condiments,Sauces,18.0,D,58.949199,C,courte paille
4,0000000001281,Tarte noix de coco,,Tartes,Tartes sucrées,Tartes à la noix de coco,14.0,D,NaN,NaN,"Crous Resto',Crous"
5,0000000001885,"Compote, tout type de fruits",,Aliments et boissons à base de végétaux,Aliments d'origine végétale,Desserts,-2.0,A,76.559933,B,Crous


In [11]:
# Reinitialisation de l'index
droppedMain_df.reset_index(drop=True, inplace=True)

In [12]:
# Suppression des produits sans lieu de manufacture

noPlace = 0
deletedIndex = []

for i in range(len(droppedMain_df)):
    if pd.isnull(droppedMain_df.loc[i,'Endroit de Manufacture']):
        deletedIndex.append(droppedMain_df.index[[i]].tolist()[0])
        noPlace += 1
    if droppedMain_df.loc[i,'Endroit de Manufacture'] in ['']:
        deletedIndex.append(droppedMain_df.index[[i]].tolist()[0])
        noPlace += 1

droppedMain_df = droppedMain_df.drop(droppedMain_df.index[deletedIndex])

print("%s produits n'avaient aucun lieu de manufacture et ont donc ete supprimes. Il reste %s produits." %(noPlace,len(droppedMain_df)))


# Reinitialisation de l'index
droppedMain_df.reset_index(drop=True, inplace=True)


droppedMain_df.head()

117464 produits n'avaient aucun lieu de manufacture et ont donc ete supprimes. Il reste 49206 produits.


,Code Barre,Nom du Produit,Endroit de Manufacture,Categorie Principale,Categorie Secondaire,Categorie Tertiaire,Nutriscore,Lettre Nutriscore,Ecoscore,Lettre Ecoscore,Marque de Distribution
0,0000000274722,Paupiette de volaille,France,Plats préparés,Produits à la viande,Plats préparés à la viande,0.0,B,51.185080,C,Comme J’aime
1,0000007730009,Sablé pâtissier,"France,Avranches",Snacks,Snacks sucrés,Biscuits et gâteaux,NaN,NaN,57.495589,C,St Michel
2,0000009336247,"Bonbons, tout type",Etats-unis,Snacks,Snacks sucrés,Confiseries,14.0,D,38.224110,D,"Wonka,Nerds"
3,0000010090206,"Thé infusé, non sucré",France,Aliments et boissons à base de végétaux,Boissons,Aliments d'origine végétale,NaN,NaN,83.954970,A,Alice Délice
4,0000020004552,Vin blanc 11°,France,Boissons,Boissons alcoolisées,Vins,NaN,NaN,102.297042,A,Union des Vignerons des Cotes du Rhône


In [13]:
# Remplacement des NaN values des categories par des chaines vides pour pouvoir comparer les categories par la suite

for i in range(len(droppedMain_df)): 
    if type(droppedMain_df.loc[i,'Categorie Principale']) == float:
        droppedMain_df.loc[i,'Categorie Principale'] = ''
    if type(droppedMain_df.loc[i,'Categorie Secondaire']) == float:
        droppedMain_df.loc[i,'Categorie Secondaire'] = ''
    if type(droppedMain_df.loc[i,'Categorie Tertiaire']) == float:
        droppedMain_df.loc[i,'Categorie Tertiaire'] = ''

droppedMain_df.head()

,Code Barre,Nom du Produit,Endroit de Manufacture,Categorie Principale,Categorie Secondaire,Categorie Tertiaire,Nutriscore,Lettre Nutriscore,Ecoscore,Lettre Ecoscore,Marque de Distribution
0,0000000274722,Paupiette de volaille,France,Plats préparés,Produits à la viande,Plats préparés à la viande,0.0,B,51.185080,C,Comme J’aime
1,0000007730009,Sablé pâtissier,"France,Avranches",Snacks,Snacks sucrés,Biscuits et gâteaux,NaN,NaN,57.495589,C,St Michel
2,0000009336247,"Bonbons, tout type",Etats-unis,Snacks,Snacks sucrés,Confiseries,14.0,D,38.224110,D,"Wonka,Nerds"
3,0000010090206,"Thé infusé, non sucré",France,Aliments et boissons à base de végétaux,Boissons,Aliments d'origine végétale,NaN,NaN,83.954970,A,Alice Délice
4,0000020004552,Vin blanc 11°,France,Boissons,Boissons alcoolisées,Vins,NaN,NaN,102.297042,A,Union des Vignerons des Cotes du Rhône


In [14]:
# On regarde la prevalence de donnees manquantes

droppedMain_df.isna().sum()

Code Barre                    0
Nom du Produit                1
Endroit de Manufacture        0
Categorie Principale          0
Categorie Secondaire          0
Categorie Tertiaire           0
Nutriscore                 7712
Lettre Nutriscore          7712
Ecoscore                  10968
Lettre Ecoscore           10968
Marque de Distribution       33
dtype: int64

In [15]:
# On regarde la diversite de certaines donnees

print('Le nombre de Categories differentes est : %s' % (len(pd.unique(droppedMain_df['Categorie Principale']))))
print('Le nombre de Lieux de Manufacture differents est : %s' % (len(pd.unique(droppedMain_df['Endroit de Manufacture']))))

Le nombre de Categories differentes est : 1047
Le nombre de Lieux de Manufacture differents est : 9576


In [16]:
# On regarde la frequence des premiers elements
print('La frequence des 20 premieres Categories est : \n%s\n\n' %(droppedMain_df["Categorie Principale"].value_counts(normalize=True).head(20)))
print('La frequence des 20 premiers Lieux est : \n%s' %(droppedMain_df["Endroit de Manufacture"].value_counts(normalize=True).head(20)))

La frequence des 20 premieres Categories est : 
Aliments et boissons à base de végétaux    0.245458
Snacks                                     0.147706
Produits laitiers                          0.126407
Viandes                                    0.109722
Boissons                                   0.069849
Plats préparés                             0.063366
Conserves                                  0.031134
Epicerie                                   0.030484
Produits de la mer                         0.027009
Surgelés                                   0.022010
Desserts                                   0.020871
Produits à tartiner                        0.018087
Sandwichs                                  0.006666
Produits d'élevages                        0.006625
Poissons et viandes et oeufs               0.005894
Plant-based foods and beverages            0.005711
Crêpes et galettes                         0.004999
Biscuits et gâteaux                        0.003943
Vinaigres       

In [17]:
# Creation d'un tableau de donnees pour les coordonnees GPS

coordonnees_df = pd.DataFrame(columns = ['Code Barre', 'Pays', 'Region', 'Latitude GPS', 'Longitude GPS'])
coordonnees_df ['Code Barre'] = droppedMain_df['Code Barre']
coordonnees_df ['Pays'] = droppedMain_df['Endroit de Manufacture'].str.lower()

for i in range(len(coordonnees_df)):
    coordonnees_df.loc[i,'Pays'] = re.sub('-',' ',str(coordonnees_df.loc[i,'Pays']))
    coordonnees_df.loc[i,'Pays'] = re.sub('_',' ',str(coordonnees_df.loc[i,'Pays']))
    coordonnees_df.loc[i,'Pays'] = re.sub("'",' ',str(coordonnees_df.loc[i,'Pays']))
    coordonnees_df.loc[i,'Pays'] = coordonnees_df.loc[i,'Pays'].split(',')
coordonnees_df.head(20)

,Code Barre,Pays,Region,Latitude GPS,Longitude GPS
0,0000000274722,[france],NaN,NaN,NaN
1,0000007730009,"[france, avranches]",NaN,NaN,NaN
2,0000009336247,[etats unis],NaN,NaN,NaN
3,0000010090206,[france],NaN,NaN,NaN
4,0000020004552,[france],NaN,NaN,NaN
5,000010,"[saint yrieix, france]",NaN,NaN,NaN
6,000030,[france],NaN,NaN,NaN
7,000031,"[france, limousin, 87500, saint yrieux]",NaN,NaN,NaN
8,000050,"[france, 87500]",NaN,NaN,NaN
9,000051,"[87500, france]",NaN,NaN,NaN


In [18]:
## Construction des regions et mise a jour de la colonne Pays
# Creation de listes pour effectuer recherches et tris sur la colonne Pays

paysList = pd.read_csv('Ressources/pays.csv', usecols = ['Nom'])['Nom'].values.tolist()
paysList = [x.lower() for x in paysList]
paysList.remove('france')
for i in range(len(paysList)):
    paysList[i] = re.sub('-',' ',str(paysList[i]))
    paysList[i] = re.sub('_',' ',str(paysList[i]))
    paysList[i] = re.sub("'",' ',str(paysList[i]))
    paysList[i] = re.sub("é",'e',str(paysList[i]))
    paysList[i] = re.sub("è",'e',str(paysList[i]))
    paysList[i] = re.sub("ê",'e',str(paysList[i]))
    paysList[i] = re.sub("î",'i',str(paysList[i]))
    paysList[i] = re.sub("ï",'i',str(paysList[i]))
    paysList[i] = re.sub("à",'a',str(paysList[i]))
    paysList[i] = re.sub("ç",'c',str(paysList[i]))
    paysList[i] = re.sub("ù",'u',str(paysList[i]))

regionList = pd.read_csv('Ressources/Geolocalisation/regions.csv', usecols = ['slug'])['slug'].values.tolist()

villeList = pd.read_csv('Ressources/Geolocalisation/cities.csv', usecols = ['slug'])['slug'].values.tolist()

codePostalList = [str(x) for x in pd.read_csv('Ressources/Geolocalisation/ville.csv', usecols = ['Code Postal'])['Code Postal'].values.tolist()]

In [19]:
# Actualisation du tableau

for i in range(len(coordonnees_df)):
# Analyse de la cellule
    if coordonnees_df.loc[i,'Pays'][0] in ['france'] and len(coordonnees_df.loc[i,'Pays']) == 1:
        coordonnees_df.loc[i,'Pays'] = 'FRANCE'
    else:
        test = [j for j in coordonnees_df.loc[i,'Pays'] if(j not in ['france'])]    

# Tri des Pays
        pays = [k for k in paysList if (k in test)]
        pays = list(set(s for s in pays if len(s) == max(len(s) for s in pays)))
        if len(pays) != 0:
            coordonnees_df.loc[i,'Pays'] = pays[0].upper()
            
#Tri des Regions
        else:
            region = [k for k in regionList if (k in test)]
            region = list(set(s for s in region if len(s) == max(len(s) for s in region))) 
            if len(region) != 0:
                coordonnees_df.loc[i,'Pays'] = 'FRANCE'
                coordonnees_df.loc[i,'Region'] = region[0]
            
#Tri des Codes Postaux  
            else:
                codePostal = [k for k in codePostalList if (k in test)]
                codePostal = list(set(s for s in codePostal if len(s) == max(len(s) for s in codePostal)))
                if len(codePostal) != 0:
                    coordonnees_df.loc[i,'Pays'] = 'FRANCE'
                    coordonnees_df.loc[i,'Region'] = codePostal[0]
            
#Tri des Villes        
                else:
                    ville = [k for k in villeList if (k in test)]
                    ville = list(set(s for s in ville if len(s) == max(len(s) for s in ville)))    
                    if len(ville) != 0:
                        coordonnees_df.loc[i,'Pays'] = 'FRANCE'
                        coordonnees_df.loc[i,'Region'] = ville[0]
                    else:
                        if np.vectorize(lambda x: bool(re.compile(r'.*france.*').match(x)))(coordonnees_df.loc[i,'Pays']).any():
                            coordonnees_df.loc[i,'Pays'] = 'FRANCE'

coordonnees_df['Pays'] = coordonnees_df['Pays'].str.title()

coordonnees_df.head(20)

,Code Barre,Pays,Region,Latitude GPS,Longitude GPS
0,0000000274722,France,NaN,NaN,NaN
1,0000007730009,France,avranches,NaN,NaN
2,0000009336247,Etats Unis,NaN,NaN,NaN
3,0000010090206,France,NaN,NaN,NaN
4,0000020004552,France,NaN,NaN,NaN
5,000010,France,NaN,NaN,NaN
6,000030,France,NaN,NaN,NaN
7,000031,France,87500,NaN,NaN
8,000050,France,87500,NaN,NaN
9,000051,France,87500,NaN,NaN


In [20]:
# Recapitulatif de la normalisation des donnees

print('nombre de Regions uniques : %s' %(len(pd.unique(coordonnees_df['Region']))))
print('nombre de Pays uniques : %s' %(len(pd.unique(coordonnees_df['Pays']))))
print("nombre de produits : %s" %len(coordonnees_df))
print(coordonnees_df.isna().sum())

nombre de Regions uniques : 1463
nombre de Pays uniques : 88
nombre de produits : 49206
Code Barre           0
Pays              4992
Region           39133
Latitude GPS     49206
Longitude GPS    49206
dtype: int64


In [21]:
# Suppression des produits sans Pays

noCountry = 0
deletedIndex = []

for i in range(len(coordonnees_df)):
    if pd.isnull(coordonnees_df.loc[i,'Pays']):
        deletedIndex.append(coordonnees_df.index[[i]].tolist()[0])
        noCountry += 1

coordonnees_df = coordonnees_df.drop(coordonnees_df.index[deletedIndex])

print("%s produits n'avaient aucun pays de manufacture et ont donc ete supprimes. Il reste %s produits." %(noCountry,len(coordonnees_df)))


# Reinitialisation de l'index
coordonnees_df.reset_index(drop=True, inplace=True)

4992 produits n'avaient aucun pays de manufacture et ont donc ete supprimes. Il reste 44214 produits.


In [22]:
## Actualisation des Regions
# Creation des tableaux necessaires
correspondanceVilleDepartement_df = pd.read_csv('Ressources/Geolocalisation/cities.csv', usecols = ['slug','department_code'])
correspondanceVilleDepartement_df = correspondanceVilleDepartement_df[['slug', 'department_code']] 
correspondanceVilleDepartement_df.columns = ['Villes', 'Numeros de Departement'] 

correspondanceDepartementRegion_df = pd.read_csv('Ressources/Geolocalisation/departments.csv', usecols = ['code','region_code'])
correspondanceDepartementRegion_df = correspondanceDepartementRegion_df[['code','region_code']] 
correspondanceDepartementRegion_df.columns = ['Numeros de Departement', 'Numeros de Region'] 

correspondanceRegion_df = pd.read_csv('Ressources/Geolocalisation/regions.csv', usecols = ['code','slug','name'])
correspondanceRegion_df = correspondanceRegion_df[['code','slug','name']] 
correspondanceRegion_df.columns = ['Numeros de Region','Noms' , 'Noms Officiels']

In [23]:
# Remplacement des Regions
for i in range(len(coordonnees_df)):
    test = [coordonnees_df.loc[i,'Region']]
    
# Correspondance des Regions
    region = [k for k in regionList if (k in test)]
    if len(region) != 0:
        region = region[0]
        coordonnees_df.loc[i,'Region'] = correspondanceRegion_df.loc[correspondanceRegion_df.index[(correspondanceRegion_df['Noms'] == region)].tolist()[0],'Noms Officiels']

# Correspondance des Codes Postaux
    else:
        codePostal = [k for k in codePostalList if (k in test)]
        if len(codePostal) != 0:
            codePostal = codePostal[0][:2]
            if codePostal == '20':
                codePostal = '2A'
            numeroRegion = correspondanceDepartementRegion_df.loc[correspondanceDepartementRegion_df.index[(correspondanceDepartementRegion_df['Numeros de Departement'] == codePostal)].tolist()[0],'Numeros de Region']
            coordonnees_df.loc[i,'Region'] = correspondanceRegion_df.loc[correspondanceRegion_df.index[(correspondanceRegion_df['Numeros de Region'] == numeroRegion)].tolist()[0],'Noms Officiels']

# Correspondance des Villes
        else:
            ville = [k for k in villeList if (k in test)]   
            if len(ville) != 0:
                ville = ville[0]
                numeroDepartement = correspondanceVilleDepartement_df.loc[correspondanceVilleDepartement_df.index[(correspondanceVilleDepartement_df['Villes'] == ville)].tolist()[0],'Numeros de Departement']
                numeroRegion = correspondanceDepartementRegion_df.loc[correspondanceDepartementRegion_df.index[(correspondanceDepartementRegion_df['Numeros de Departement'] == numeroDepartement)].tolist()[0],'Numeros de Region']
                coordonnees_df.loc[i,'Region'] = correspondanceRegion_df.loc[correspondanceRegion_df.index[(correspondanceRegion_df['Numeros de Region'] == numeroRegion)].tolist()[0],'Noms Officiels']

coordonnees_df.head(20)

,Code Barre,Pays,Region,Latitude GPS,Longitude GPS
0,0000000274722,France,NaN,NaN,NaN
1,0000007730009,France,Normandie,NaN,NaN
2,0000009336247,Etats Unis,NaN,NaN,NaN
3,0000010090206,France,NaN,NaN,NaN
4,0000020004552,France,NaN,NaN,NaN
5,000010,France,NaN,NaN,NaN
6,000030,France,NaN,NaN,NaN
7,000031,France,Nouvelle-Aquitaine,NaN,NaN
8,000050,France,Nouvelle-Aquitaine,NaN,NaN
9,000051,France,Nouvelle-Aquitaine,NaN,NaN


In [24]:
# Verification de l'homogeneisation des Regions

print('nombre de Regions uniques : %s' %(len(pd.unique(coordonnees_df['Region']))))
print(coordonnees_df.isna().sum())

nombre de Regions uniques : 17
Code Barre           0
Pays                 0
Region           34141
Latitude GPS     44214
Longitude GPS    44214
dtype: int64


In [25]:
## Remplissage des coordonnees GPS
# Constructions de dictionnaires pour remplir le tableau

# Pour les Regions
chefLieuxDict = dict(zip(pd.read_csv('Ressources/Geolocalisation/chefLieux.csv', usecols = ['Région'])['Région'].tolist(),pd.read_csv('Ressources/Geolocalisation/chefLieux.csv', usecols = ['Chef-lieu de région'])['Chef-lieu de région'].tolist()))

regionGPS_df = pd.read_csv('Ressources/Geolocalisation/cities.csv', usecols = ['name','gps_lat','gps_lng'])
regionGPS_df = regionGPS_df[regionGPS_df['name'].isin(list(chefLieuxDict.values()))]
regionGPS_df.sort_values("name", inplace = True) 
regionGPS_df.drop_duplicates(subset ="name", keep = 'first', inplace = True) 
regionGPS_df.reset_index(drop=True, inplace=True)

latitudeDict = {}
longitudeDict = {}
for region in list(chefLieuxDict.keys()):
    latitudeDict [region]= regionGPS_df.loc[regionGPS_df.index[(regionGPS_df['name'] == chefLieuxDict[region])].tolist()[0],'gps_lat']
    longitudeDict [region]= regionGPS_df.loc[regionGPS_df.index[(regionGPS_df['name'] == chefLieuxDict[region])].tolist()[0],'gps_lng']

# Pour les Pays 
code2List = pd.read_csv('Ressources/pays.csv', usecols = ['Standard 2'])['Standard 2'].tolist()
code2List.remove('FR')
paysList = [element.title() for element in paysList]
paysDict = dict(zip(paysList,code2List))

droppedKey = []
for key, value in paysDict.items():
    if key not in list(pd.unique(coordonnees_df['Pays'])):
        droppedKey.append(key)

for key in droppedKey:
    del paysDict[key]
del paysDict['Namibie']    

paysGPS_df = pd.read_csv('Ressources/Geolocalisation/countries.csv', usecols = ['country','latitude','longitude'])
paysGPS_df = paysGPS_df[paysGPS_df['country'].isin(list(paysDict.values()))]
paysGPS_df.sort_values("country", inplace = True) 
paysGPS_df.drop_duplicates(subset ="country", keep = 'first', inplace = True) 
paysGPS_df.reset_index(drop=True, inplace=True)
paysGPS_df.dropna()

droppedKey = []
for key, value in paysDict.items():
    if value not in list(pd.unique(paysGPS_df['country'])):
        droppedKey.append(key)

for key in droppedKey:
    del paysDict[key]

for pays in list(paysDict.keys()):
    latitudeDict [pays] = paysGPS_df.loc[paysGPS_df.index[(paysGPS_df['country'] == paysDict[pays])].tolist()[0],'latitude']
    longitudeDict [pays] = paysGPS_df.loc[paysGPS_df.index[(paysGPS_df['country'] == paysDict[pays])].tolist()[0],'longitude']

In [26]:
# Remplissage des colonnes lat et long

for i in range(len(coordonnees_df)):
# Pour les Regions
    test = [coordonnees_df.loc[i,'Region']]
    region = [k for k in list(chefLieuxDict.keys()) if (k in test)]
    if len(region) != 0:
        region = region[0]
        coordonnees_df.loc[i,'Latitude GPS'] = latitudeDict [region]
        coordonnees_df.loc[i,'Longitude GPS'] = longitudeDict [region]
# Pour les Pays
    test = [coordonnees_df.loc[i,'Pays']]
    pays = [k for k in list(paysDict.keys()) if (k in test)]
    if len(pays) != 0:
        pays = pays[0]
        coordonnees_df.loc[i,'Latitude GPS'] = latitudeDict [pays]
        coordonnees_df.loc[i,'Longitude GPS'] = longitudeDict [pays]
        
coordonnees_df.head(20)

,Code Barre,Pays,Region,Latitude GPS,Longitude GPS
0,0000000274722,France,NaN,NaN,NaN
1,0000007730009,France,Normandie,49.434,1.07759
2,0000009336247,Etats Unis,NaN,37.09024,-95.712891
3,0000010090206,France,NaN,NaN,NaN
4,0000020004552,France,NaN,NaN,NaN
5,000010,France,NaN,NaN,NaN
6,000030,France,NaN,NaN,NaN
7,000031,France,Nouvelle-Aquitaine,44.835,-0.587269
8,000050,France,Nouvelle-Aquitaine,44.835,-0.587269
9,000051,France,Nouvelle-Aquitaine,44.835,-0.587269


In [27]:
# Construction du tableau recapitulatif final

finalMain_df = pd.DataFrame(columns = ['Code Barre','Nom du Produit','Pays de Manufacture', 'Region de Manufacture','Latitude GPS','Longitude GPS','Categorie Principale',
                                       'Categorie Secondaire','Categorie Tertiaire','Nutriscore','Lettre Nutriscore','Ecoscore','Lettre Ecoscore','Marque de Distribution'])

finalMain_df['Code Barre'] = coordonnees_df['Code Barre']
finalMain_df['Pays de Manufacture'] = coordonnees_df['Pays']
finalMain_df['Region de Manufacture'] = coordonnees_df['Region']
finalMain_df['Latitude GPS'] = coordonnees_df['Latitude GPS']
finalMain_df['Longitude GPS'] = coordonnees_df['Longitude GPS']

mddList = ['Boni Selection', 'Boni', 'U', 'Intermarché', 'Auchan', 'Lidl','Franprix', 'Casino', 'Monoprix', 'Carrefour']

for i in range(len(finalMain_df)):
    codeBarre = finalMain_df.loc[i,'Code Barre']
    droppedMainIndex = droppedMain_df.index[(droppedMain_df['Code Barre'] == codeBarre)].tolist()[0]
    finalMain_df.loc[i,'Nom du Produit'] = droppedMain_df.loc[droppedMainIndex,'Nom du Produit']
    finalMain_df.loc[i,'Categorie Principale'] = droppedMain_df.loc[droppedMainIndex,'Categorie Principale']
    finalMain_df.loc[i,'Categorie Secondaire'] = droppedMain_df.loc[droppedMainIndex,'Categorie Secondaire']
    finalMain_df.loc[i,'Categorie Tertiaire'] = droppedMain_df.loc[droppedMainIndex,'Categorie Tertiaire']
    finalMain_df.loc[i,'Nutriscore'] = droppedMain_df.loc[droppedMainIndex,'Nutriscore']
    finalMain_df.loc[i,'Lettre Nutriscore'] = droppedMain_df.loc[droppedMainIndex,'Lettre Nutriscore']
    finalMain_df.loc[i,'Ecoscore'] = droppedMain_df.loc[droppedMainIndex,'Ecoscore']
    finalMain_df.loc[i,'Lettre Ecoscore'] = droppedMain_df.loc[droppedMainIndex,'Lettre Ecoscore']
    if droppedMain_df.loc[droppedMainIndex,'Marque de Distribution'] in mddList:
        finalMain_df.loc[i,'Marque de Distribution'] = 'Oui'
    else:
        finalMain_df.loc[i,'Marque de Distribution'] = 'Non'

print(len(finalMain_df), '\n', finalMain_df.head(), '\n', finalMain_df.isna().sum())

44214 
       Code Barre         Nom du Produit Pays de Manufacture  \
0  0000000274722  Paupiette de volaille              France   
1  0000007730009        Sablé pâtissier              France   
2  0000009336247     Bonbons, tout type          Etats Unis   
3  0000010090206  Thé infusé, non sucré              France   
4  0000020004552          Vin blanc 11°              France   

  Region de Manufacture Latitude GPS Longitude GPS  \
0                   NaN          NaN           NaN   
1             Normandie       49.434       1.07759   
2                   NaN     37.09024    -95.712891   
3                   NaN          NaN           NaN   
4                   NaN          NaN           NaN   

                      Categorie Principale   Categorie Secondaire  \
0                           Plats préparés   Produits à la viande   
1                                   Snacks          Snacks sucrés   
2                                   Snacks          Snacks sucrés   
3  Aliments 

In [28]:
finalMain_df['Marque de Distribution'].value_counts(sort = True)

Non    39929
Oui     4285
Name: Marque de Distribution, dtype: int64

In [29]:
# Suppression des produits sans Noms

nanCount = 0
nanIndex = []

for i in range(len(finalMain_df)):
    if type(finalMain_df.loc[i,'Nom du Produit']) == float:
        nanIndex.append(i)
        nanCount += 1
        
finalMain_df = finalMain_df.drop(finalMain_df.index[nanIndex])

print("%s produits n'avaient aucun noms et ont donc ete supprimes. Il reste %s produits." %(nanCount,len(finalMain_df)))

# Reinitialisation de l'index
finalMain_df.reset_index(drop=True, inplace=True)

0 produits n'avaient aucun noms et ont donc ete supprimes. Il reste 44214 produits.


In [30]:
# Fonction pour normaliser le texte (enlever les accents, etc)

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

In [31]:
# Normalisation de colonnes avec beaucoup de chaines de characteres diverses

for i in range(len(finalMain_df)):    
    if type(finalMain_df.loc[i,'Nom du Produit']) == str:
        finalMain_df.loc[i,'Nom du Produit'] = remove_accents(finalMain_df.loc[i,'Nom du Produit']).decode("utf-8")
        
    if type(finalMain_df.loc[i,'Categorie Principale']) == str:
        finalMain_df.loc[i,'Categorie Principale'] = remove_accents(finalMain_df.loc[i,'Categorie Principale']).decode("utf-8")
        
    if type(finalMain_df.loc[i,'Categorie Secondaire']) == str:
        finalMain_df.loc[i,'Categorie Secondaire'] = remove_accents(finalMain_df.loc[i,'Categorie Secondaire']).decode("utf-8")
        
    if type(finalMain_df.loc[i,'Categorie Tertiaire']) == str:
        finalMain_df.loc[i,'Categorie Tertiaire'] = remove_accents(finalMain_df.loc[i,'Categorie Tertiaire']).decode("utf-8")

In [32]:
# Enregistrements des differentes etapes pour les reutiliser dans un autre Notebook si besoin

neatMain_df.to_csv('Ressources'+'/'+'01_Toutes les Donnees.csv', index = False)
droppedMain_df.to_csv('Ressources'+'/'+'02_Donnees Avec au Moins un Scores.csv', index = False)
finalMain_df.to_csv('Ressources'+'/'+'03_Donnees Nettoyees.csv', index = False)

# time needed = 20min environ